In [28]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score

In [29]:
!pip install torch torchvision


In [30]:
# Load dataset
file_path = '/content/BenchmarkUddinSO-ConsoliatedAspectSentiment.xls'  # Update this to your file path
data = pd.read_excel(file_path)
df = data[['sent', 'ManualLabel', 'codes']]  # Adjust column names as per your dataset

# Filter for usability related entries
df_usability = df[df['codes'].str.contains('OnlySentiment', case=False)]

# Prepare labels for binary classification
df_usability['ManualLabel'] = df_usability['ManualLabel'].apply(lambda x: 1 if x == 'p' else 0)

# Split the dataset
train, test = train_test_split(df_usability, test_size=0.4, random_state=42)  # Updated test_size to 40%

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(train['ManualLabel']), y=train['ManualLabel'])
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)


<ipython-input-30-6c80e1ab1c81>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_usability['ManualLabel'] = df_usability['ManualLabel'].apply(lambda x: 1 if x == 'p' else 0)


In [31]:
# Tokenization
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings = tokenizer(train['sent'].tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test['sent'].tolist(), truncation=True, padding=True, max_length=512)

train_dataset = SentimentDataset(train_encodings, train['ManualLabel'].tolist())
test_dataset = SentimentDataset(test_encodings, test['ManualLabel'].tolist())

In [32]:
!pip install transformers[torch]

In [33]:
!pip install accelerate -U


In [34]:
import torch
from torch.utils.data import DataLoader
from transformers import RobertaForSequenceClassification, RobertaTokenizer
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, f1_score

# Load the tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

# Tokenization and dataset preparation
# Assuming `train` and `test` are your pre-split DataFrame
train_encodings = tokenizer(train['sent'].tolist(), truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test['sent'].tolist(), truncation=True, padding=True, max_length=512)

train_dataset = SentimentDataset(train_encodings, train['ManualLabel'].tolist())
test_dataset = SentimentDataset(test_encodings, test['ManualLabel'].tolist())

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(3):  # Number of epochs
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


Epoch 1, Loss: 0.6722835302352905
Epoch 2, Loss: 0.6323209404945374
Epoch 3, Loss: 0.6203868389129639


In [36]:
import time

# Evaluation loop
model.eval()
predictions, true_labels = [], []
total_inference_time = 0
total_samples = 0

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        start_time = time.time()
        outputs = model(input_ids, attention_mask=attention_mask)
        inference_time = time.time() - start_time
        total_inference_time += inference_time
        total_samples += input_ids.size(0)

        logits = outputs.logits
        preds = logits.argmax(dim=1).cpu().numpy()
        labels = batch['labels'].cpu().numpy()

        predictions.extend(preds)
        true_labels.extend(labels)

# Compute metrics
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

print(f'Accuracy: {accuracy:.4f}')
print(f'F1 Score: {f1:.4f}')

# Calculate average inference time per sample
average_inference_time = total_inference_time / total_samples
print(f'Total Inference Time: {total_inference_time:.6f} seconds')
print(f'Total Samples: {total_samples}')
print(f'Average Inference Time per Sample: {average_inference_time:.6f} seconds')


Accuracy: 0.6357
F1 Score: 0.4848
Total Inference Time: 25.509454 seconds
Total Samples: 140
Average Inference Time per Sample: 0.182210 seconds
